###1: Setup and Mount Drive

In [1]:
import os
import zipfile
import shutil
import pandas as pd
import numpy as np
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# Define Paths
BASE_PATH = '/content/drive/MyDrive/Statistella_Submission'
RAW_ZIP_PATH = os.path.join(BASE_PATH, '00_Source_Zipped')
EXTRACT_PATH = os.path.join(BASE_PATH, '01_Raw_Extracted')
PROCESSED_PATH = os.path.join(BASE_PATH, '02_Processed_Data')

# Create Directories
os.makedirs(EXTRACT_PATH, exist_ok=True)
os.makedirs(PROCESSED_PATH, exist_ok=True)

print("Setup Complete.")

Mounted at /content/drive
Setup Complete.


###2: Extraction Engine

In [ ]:
print("Starting Extraction...")

# Clear extraction folder
for f in os.listdir(EXTRACT_PATH):
    path = os.path.join(EXTRACT_PATH, f)
    if os.path.isfile(path):
        os.remove(path)

# Iterate through Source files
files_found = os.listdir(RAW_ZIP_PATH)
for filename in files_found:
    src_path = os.path.join(RAW_ZIP_PATH, filename)

    # Unzip .zip files
    if filename.endswith('.zip'):
        try:
            with zipfile.ZipFile(src_path, 'r') as zip_ref:
                zip_ref.extractall(EXTRACT_PATH)
                print(f"Unzipped: {filename}")
        except:
            print(f"Warning: Failed to unzip {filename}")

    # Copy .csv files
    elif filename.endswith('.csv'):
        shutil.copy(src_path, os.path.join(EXTRACT_PATH, filename))
        print(f"Copied: {filename}")

print("Extraction Complete.")

Starting Extraction...
Unzipped: ranking.csv.zip
Copied: teams.csv
Copied: players.csv
Unzipped: games.csv.zip
Unzipped: games_details.csv.zip
Extraction Complete.


###3: Loading and Cleaning

In [2]:
print("Inspecting CSV files and their columns...\n")

files = [
    "games.csv",
    "teams.csv",
    "players.csv",
    "ranking.csv",
    "games_details.csv"
]

for file in files:
    path = os.path.join(EXTRACT_PATH, file)
    df = pd.read_csv(path, nrows=5, low_memory=False)
    print(f"File: {file}")
    print("Columns:")
    for col in df.columns:
        print(f"  - {col}")
    print("-" * 50)


Inspecting CSV files and their columns...

File: games.csv
Columns:
  - GAME_DATE_EST
  - GAME_ID
  - GAME_STATUS_TEXT
  - HOME_TEAM_ID
  - VISITOR_TEAM_ID
  - SEASON
  - TEAM_ID_home
  - PTS_home
  - FG_PCT_home
  - FT_PCT_home
  - FG3_PCT_home
  - AST_home
  - REB_home
  - TEAM_ID_away
  - PTS_away
  - FG_PCT_away
  - FT_PCT_away
  - FG3_PCT_away
  - AST_away
  - REB_away
  - HOME_TEAM_WINS
--------------------------------------------------
File: teams.csv
Columns:
  - LEAGUE_ID
  - TEAM_ID
  - MIN_YEAR
  - MAX_YEAR
  - ABBREVIATION
  - NICKNAME
  - YEARFOUNDED
  - CITY
  - ARENA
  - ARENACAPACITY
  - OWNER
  - GENERALMANAGER
  - HEADCOACH
  - DLEAGUEAFFILIATION
--------------------------------------------------
File: players.csv
Columns:
  - PLAYER_NAME
  - TEAM_ID
  - PLAYER_ID
  - SEASON
--------------------------------------------------
File: ranking.csv
Columns:
  - TEAM_ID
  - LEAGUE_ID
  - SEASON_ID
  - STANDINGSDATE
  - CONFERENCE
  - TEAM
  - G
  - W
  - L
  - W_PCT
  - HOME

In [3]:
files = [
    "games.csv",
    "teams.csv",
    "players.csv",
    "ranking.csv",
    "games_details.csv"
]

for file in files:
    print(f"\n===== {file} =====")
    df = pd.read_csv(os.path.join(EXTRACT_PATH, file), low_memory=False)
    print(df.head(5))


===== games.csv =====
  GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0    2022-12-22  22200477            Final    1610612740       1610612759   
1    2022-12-22  22200478            Final    1610612762       1610612764   
2    2022-12-21  22200466            Final    1610612739       1610612749   
3    2022-12-21  22200467            Final    1610612755       1610612765   
4    2022-12-21  22200468            Final    1610612737       1610612741   

   SEASON  TEAM_ID_home  PTS_home  FG_PCT_home  FT_PCT_home  ...  AST_home  \
0    2022    1610612740     126.0        0.484        0.926  ...      25.0   
1    2022    1610612762     120.0        0.488        0.952  ...      16.0   
2    2022    1610612739     114.0        0.482        0.786  ...      22.0   
3    2022    1610612755     113.0        0.441        0.909  ...      27.0   
4    2022    1610612737     108.0        0.429        1.000  ...      22.0   

   REB_home  TEAM_ID_away  PTS_away  FG_PCT_a

In [5]:
print("Loading Data...")

# 1. Load Games
games = pd.read_csv(os.path.join(EXTRACT_PATH, 'games.csv'))
games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'])
games = games[games['SEASON'] >= 2004] # Filter Post-2004
print(f"Games loaded: {games.shape[0]} rows")

# 2. Load Teams
teams = pd.read_csv(os.path.join(EXTRACT_PATH, 'teams.csv'))
print(f"Teams loaded: {teams.shape[0]} rows")

# 3. Load Ranking (CRITICAL: To get Conference info)
ranking = pd.read_csv(os.path.join(EXTRACT_PATH, 'ranking.csv'))
# We only need a unique mapping of TEAM_ID -> CONFERENCE
# We take the latest entry for each team to verify their conference
conference_map = ranking.drop_duplicates(subset='TEAM_ID', keep='last')[['TEAM_ID', 'CONFERENCE']]
print(f"Conference Mappings loaded: {conference_map.shape[0]} teams")

# 4. Load Player Details
details = pd.read_csv(os.path.join(EXTRACT_PATH, 'games_details.csv'), low_memory=False)
print(f"Details loaded: {details.shape[0]} rows")

Loading Data...
Games loaded: 25266 rows
Teams loaded: 30 rows
Conference Mappings loaded: 30 teams
Details loaded: 668628 rows


###4: The Master Merge

In [ ]:
print("Merging Datasets...")

# Optimization: Filter details for only the valid games
valid_ids = games['GAME_ID'].unique()
details = details[details['GAME_ID'].isin(valid_ids)]

# Merge 1: Player Stats + Game Date
master_df = pd.merge(
    details,
    games[['GAME_ID', 'GAME_DATE_EST', 'SEASON', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'HOME_TEAM_WINS']],
    on='GAME_ID',
    how='inner'
)

# Merge 2: Add Team Name/City
master_df = pd.merge(
    master_df,
    teams[['TEAM_ID', 'ABBREVIATION', 'NICKNAME', 'CITY']],
    on='TEAM_ID',
    how='left'
)

# Merge 3: Add Conference Info (Crucial for East vs West analysis)
master_df = pd.merge(
    master_df,
    conference_map,
    on='TEAM_ID',
    how='left'
)

print(f"Final Data Shape: {master_df.shape}")

Merging Datasets...
Final Data Shape: (638857, 38)


## A. team_game_stats

This dataset represents one team’s performance in a single game.  
Each NBA game appears twice in this table: once for the home team and once for the away team.

**What this dataset enables**
- Season-wise and team-wise scoring trends
- Home vs Away performance comparison
- Eastern vs Western Conference analysis
- Team consistency and win–loss patterns over time

This table is the backbone for all team-level and trend-based visualizations in the dashboard.

In [6]:
# Step 1: Prepare home team records from games table
home_df = games[[
    "GAME_ID", "GAME_DATE_EST", "SEASON",
    "HOME_TEAM_ID", "VISITOR_TEAM_ID",
    "PTS_home", "FG_PCT_home", "FT_PCT_home", "FG3_PCT_home",
    "AST_home", "REB_home", "HOME_TEAM_WINS"
]].copy()

home_df["TEAM_ID"] = home_df["HOME_TEAM_ID"]
home_df["OPPONENT_TEAM_ID"] = home_df["VISITOR_TEAM_ID"]
home_df["PTS"] = home_df["PTS_home"]
home_df["FG_PCT"] = home_df["FG_PCT_home"]
home_df["FT_PCT"] = home_df["FT_PCT_home"]
home_df["FG3_PCT"] = home_df["FG3_PCT_home"]
home_df["AST"] = home_df["AST_home"]
home_df["REB"] = home_df["REB_home"]
home_df["HOME_AWAY"] = "Home"
home_df["WIN"] = home_df["HOME_TEAM_WINS"]

# Step 2: Prepare away team records from games table
away_df = games[[
    "GAME_ID", "GAME_DATE_EST", "SEASON",
    "HOME_TEAM_ID", "VISITOR_TEAM_ID",
    "PTS_away", "FG_PCT_away", "FT_PCT_away", "FG3_PCT_away",
    "AST_away", "REB_away", "HOME_TEAM_WINS"
]].copy()

away_df["TEAM_ID"] = away_df["VISITOR_TEAM_ID"]
away_df["OPPONENT_TEAM_ID"] = away_df["HOME_TEAM_ID"]
away_df["PTS"] = away_df["PTS_away"]
away_df["FG_PCT"] = away_df["FG_PCT_away"]
away_df["FT_PCT"] = away_df["FT_PCT_away"]
away_df["FG3_PCT"] = away_df["FG3_PCT_away"]
away_df["AST"] = away_df["AST_away"]
away_df["REB"] = away_df["REB_away"]
away_df["HOME_AWAY"] = "Away"
away_df["WIN"] = 1 - away_df["HOME_TEAM_WINS"]

# Step 3: Combine home and away into one team-game table
team_game_stats = pd.concat([home_df, away_df], ignore_index=True)

# Step 4: Keep only standardized columns
team_game_stats = team_game_stats[[
    "GAME_ID", "GAME_DATE_EST", "SEASON",
    "TEAM_ID", "OPPONENT_TEAM_ID",
    "HOME_AWAY", "WIN",
    "PTS", "FG_PCT", "FT_PCT", "FG3_PCT", "AST", "REB"
]]

# Step 5: Add team metadata
team_game_stats = team_game_stats.merge(
    teams[["TEAM_ID", "ABBREVIATION", "NICKNAME", "CITY"]],
    on="TEAM_ID",
    how="left"
)

# Step 6: Add conference information
team_game_stats = team_game_stats.merge(
    conference_map,
    on="TEAM_ID",
    how="left"
)

# Step 7: Final sanity check
print("team_game_stats shape:", team_game_stats.shape)
team_game_stats.head()


team_game_stats shape: (50532, 17)


,GAME_ID,GAME_DATE_EST,SEASON,TEAM_ID,OPPONENT_TEAM_ID,HOME_AWAY,WIN,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,ABBREVIATION,NICKNAME,CITY,CONFERENCE
0,22200477,2022-12-22,2022,1610612740,1610612759,Home,1,126.0,0.484,0.926,0.382,25.0,46.0,NOP,Pelicans,New Orleans,East
1,22200478,2022-12-22,2022,1610612762,1610612764,Home,1,120.0,0.488,0.952,0.457,16.0,40.0,UTA,Jazz,Utah,West
2,22200466,2022-12-21,2022,1610612739,1610612749,Home,1,114.0,0.482,0.786,0.313,22.0,37.0,CLE,Cavaliers,Cleveland,East
3,22200467,2022-12-21,2022,1610612755,1610612765,Home,1,113.0,0.441,0.909,0.297,27.0,49.0,PHI,76ers,Philadelphia,East
4,22200468,2022-12-21,2022,1610612737,1610612741,Home,0,108.0,0.429,1.000,0.378,22.0,47.0,ATL,Hawks,Atlanta,East


In [8]:
# Add point differential
team_game_stats["POINT_DIFF"] = team_game_stats["PTS"] - team_game_stats.groupby("GAME_ID")["PTS"].transform("sum") + team_game_stats["PTS"]

# Add readable game result
team_game_stats["GAME_RESULT"] = team_game_stats["WIN"].map({1: "Win", 0: "Loss"})

# Add binary home flag
team_game_stats["IS_HOME"] = (team_game_stats["HOME_AWAY"] == "Home").astype(int)

# Add scoring category
team_game_stats["SCORING_TYPE"] = team_game_stats["PTS"].apply(
    lambda x: "High Scoring" if x >= 110 else "Low Scoring"
)

# Add season phase
team_game_stats["SEASON_PHASE"] = team_game_stats["GAME_DATE_EST"].dt.month.apply(
    lambda x: "Early Season" if x <= 12 else "Late Season"
)

## B. player_game_stats

This dataset represents one player’s performance in a single game, captured at the finest level of detail available.

**What this dataset enables**
- Player scoring, efficiency, and impact analysis
- Consistency analysis across games and seasons
- Identification of standout and high-impact performances
- Player-level filtering and interactive exploration

This table powers all player-focused insights while keeping team-level analysis clean and accurate.


In [7]:
# Step 1: Select core player performance columns
player_game_stats = details[[
    "GAME_ID", "TEAM_ID", "PLAYER_ID", "PLAYER_NAME",
    "MIN", "PTS", "AST", "REB", "STL", "BLK", "TO",
    "FG_PCT", "FG3_PCT", "FT_PCT", "PLUS_MINUS"
]].copy()

# Step 2: Attach game date and season from games table
player_game_stats = player_game_stats.merge(
    games[["GAME_ID", "GAME_DATE_EST", "SEASON"]],
    on="GAME_ID",
    how="left"
)

# Step 3: Attach team metadata
player_game_stats = player_game_stats.merge(
    teams[["TEAM_ID", "ABBREVIATION", "NICKNAME", "CITY"]],
    on="TEAM_ID",
    how="left"
)

# Step 4: Remove rows with no minutes played
player_game_stats = player_game_stats[player_game_stats["MIN"].notna()]

# Step 5: Final sanity check
print("player_game_stats shape:", player_game_stats.shape)
player_game_stats.head()


player_game_stats shape: (559678, 20)


,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,MIN,PTS,AST,REB,STL,BLK,TO,FG_PCT,FG3_PCT,FT_PCT,PLUS_MINUS,GAME_DATE_EST,SEASON,ABBREVIATION,NICKNAME,CITY
0,22200477,1610612759,1629641,Romeo Langford,18:06,2.0,0.0,2.0,1.0,0.0,2.0,1.000,0.000,0.0,-2.0,2022-12-22,2022.0,SAS,Spurs,San Antonio
1,22200477,1610612759,1631110,Jeremy Sochan,31:01,23.0,6.0,9.0,1.0,0.0,2.0,0.500,0.500,0.7,-14.0,2022-12-22,2022.0,SAS,Spurs,San Antonio
2,22200477,1610612759,1627751,Jakob Poeltl,21:42,13.0,1.0,4.0,1.0,0.0,2.0,0.667,0.000,1.0,-4.0,2022-12-22,2022.0,SAS,Spurs,San Antonio
3,22200477,1610612759,1630170,Devin Vassell,30:20,10.0,5.0,9.0,3.0,0.0,2.0,0.308,0.167,1.0,-18.0,2022-12-22,2022.0,SAS,Spurs,San Antonio
4,22200477,1610612759,1630200,Tre Jones,27:44,19.0,3.0,2.0,0.0,0.0,2.0,0.583,0.333,1.0,0.0,2022-12-22,2022.0,SAS,Spurs,San Antonio


In [9]:
# Convert minutes to numeric
player_game_stats["MINUTES_PLAYED"] = (
    player_game_stats["MIN"].str.split(":").str[0].astype(float) +
    player_game_stats["MIN"].str.split(":").str[1].astype(float) / 60
)

# Points per minute
player_game_stats["PTS_PER_MIN"] = player_game_stats["PTS"] / player_game_stats["MINUTES_PLAYED"]

# Simple efficiency score
player_game_stats["EFFICIENCY_SCORE"] = (
    player_game_stats["PTS"] +
    player_game_stats["REB"] +
    player_game_stats["AST"] +
    player_game_stats["STL"] +
    player_game_stats["BLK"] -
    player_game_stats["TO"]
)

# Consistency flag
player_game_stats["DOUBLE_DIGIT_PTS"] = (player_game_stats["PTS"] >= 10).astype(int)


In [10]:
print(team_game_stats.shape)
print(player_game_stats.shape)

(50532, 22)
(559678, 24)


###5: Advanced Feature Engineering

In [ ]:
print("Calculating Metrics...")

# Convert numeric columns safely
cols = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'TO', 'FGA', 'FGM', 'FTA', 'FTM']
for c in cols:
    master_df[c] = pd.to_numeric(master_df[c], errors='coerce').fillna(0)

# Efficiency Formula
master_df['EFFICIENCY'] = (
    (master_df['PTS'] + master_df['REB'] + master_df['AST'] + master_df['STL'] + master_df['BLK']) -
    ((master_df['FGA'] - master_df['FGM']) + (master_df['FTA'] - master_df['FTM']) + master_df['TO'])
)

# True Shooting %
master_df['TS_PERCENT'] = master_df['PTS'] / (2 * (master_df['FGA'] + 0.44 * master_df['FTA']))
master_df['TS_PERCENT'] = master_df['TS_PERCENT'].fillna(0)

# Win Flag
# Logic: You win if (Team is Home & HomeWins) OR (Team is Visitor & HomeLoses)
master_df['IS_WIN'] = np.where(
    (master_df['TEAM_ID'] == master_df['HOME_TEAM_ID']) & (master_df['HOME_TEAM_WINS'] == 1), 1,
    np.where(
        (master_df['TEAM_ID'] != master_df['HOME_TEAM_ID']) & (master_df['HOME_TEAM_WINS'] == 0), 1, 0
    )
)

print("Metrics Added: EFFICIENCY, TS_PERCENT, IS_WIN")

Calculating Metrics...
Metrics Added: EFFICIENCY, TS_PERCENT, IS_WIN


###6: Save to Drive

In [ ]:
output_file = os.path.join(PROCESSED_PATH, 'master_nba_data.csv')
master_df = master_df.sort_values(by='GAME_DATE_EST', ascending=False)
master_df.to_csv(output_file, index=False)
print("PIPELINE COMPLETE. Data ready for Dashboard.")

PIPELINE COMPLETE. Data ready for Dashboard.


In [11]:
# Step 1: Create processed data directory if it does not exist
os.makedirs(PROCESSED_PATH, exist_ok=True)

# Step 2: Export team-level game statistics
team_game_stats.to_csv(
    os.path.join(PROCESSED_PATH, "team_game_stats.csv"),
    index=False
)

# Step 3: Export player-level game statistics
player_game_stats.to_csv(
    os.path.join(PROCESSED_PATH, "player_game_stats.csv"),
    index=False
)

# Step 4: Confirm export completion
print("Export completed:")
print(" - team_game_stats.csv")
print(" - player_game_stats.csv")

Export completed:
 - team_game_stats.csv
 - player_game_stats.csv
